In [1]:
import numpy as np
from keras.datasets import cifar10
from keras import models
from keras import layers
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras.engine.training import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten,BatchNormalization,Activation,Conv2D,MaxPool2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
model = Sequential()
import pandas as pd
from sklearn import preprocessing

In [2]:
dataset = pd.read_csv('balance-scale.data',header=None);
training_data = dataset.sample(frac=0.90, random_state=25)
testing_data = dataset.drop(training_data.index)
validation_data = training_data[:80];
training_data = training_data[80:]
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")
print(f"No. of validation examples: {validation_data.shape[0]}")
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
training_labels = encoder.fit_transform(training_data[[0]])
training_data.pop(0);
testing_labels = encoder.fit_transform(testing_data[[0]])
testing_data.pop(0);
validation_label = encoder.fit_transform(validation_data[[0]])
validation_data.pop(0);
np.squeeze(np.array(training_labels,dtype=float))
training_data.shape[1]

No. of training examples: 482
No. of testing examples: 63
No. of validation examples: 80


4

In [179]:
import itertools
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import itertools
class ComputationGraphs:
  def __init__(self, x, y,node1=2,node2=2):
    self.X = x;
    self.Y = y
    self.ypred = np.zeros((1,self.Y.shape[1]))
    self.noOfLayers = 3
    self.param = {}
    self.ch = {}
    self.grad = {}
    self.loss = []
    self.node1 = node1
    self.node2 = node2
    self.learningrate = 0.1
    self.threshold = 0.5
  def nInit(self):
    self.param['W1'] = np.ones((self.node1,self.X.shape[1]))*0.03
    self.param['W2'] = np.ones((self.node2,self.node1))*0.03
    self.param['W3'] = np.ones((self.Y.shape[1],self.node2))*0.03
    self.param['b1'] = np.ones((self.node1,1))*0.03
    self.param['b2'] = np.ones((self.node2,1))*0.03
    self.param['b3'] = np.ones((self.Y.shape[1],1))*0.03
  def Sigmoid(self,Z):
    return 1/(1+np.exp(-Z))
  def DerivativeSigmoid(self,Z):
    h = 1/(1+np.exp(-Z))
    dZ = h * (1-h)
    return dZ
  def Softmax(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1,
                                            keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1,
                                            keepdims=True)

        return probabilities;
  def DerivateSoftmax(self,x):
    x = np.exp(x - np.max(x))
    return (np.sum(x)-x)/ np.sum(x)   
  def backwardCategoricalCrossentropy(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        dinputs = -y_true / dvalues
        dinputs = dinputs / samples
        return dinputs;
  def CategoricalCrossentropy(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped * y_true,
                axis=1
            )
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
  def forwardpass(self):
    Z1 = self.param['W1'].dot(self.X.T)
    Z1 = Z1 + self.param['b1']
    A1 = self.Sigmoid(Z1)
    self.ch['Z1'],self.ch['A1'] = Z1, A1

    Z2 = self.param['W2'].dot(A1) + self.param['b2']
    A2 = self.Sigmoid(Z2)
    self.ch['Z2'],self.ch['A2'] = Z2, A2

    Z3 = self.param['W3'].dot(A2) + self.param['b3']
    A3 = self.Softmax(Z3)
    self.ch['Z3'],self.ch['A3'] = Z3, A3
    self.ypred = A3
    loss = self.CategoricalCrossentropy(self.ypred.T,self.Y)
    return self.ypred,loss
  def backwardpass(self):
    DYpred = self.backwardCategoricalCrossentropy(self.ypred.T , self.Y)
    DZ3 = DYpred *self.DerivateSoftmax(self.ch['Z3']).T
    DA2 = np.dot(self.param['W3'].T,DZ3.T);
    DW3 = 1./self.ch['A2'].shape[1] * np.dot(DZ3.T,self.ch['A2'].T)
    DZ2 = DA2 * self.DerivativeSigmoid( self.ch['Z2'])
    DA1 = np.dot(self.param['W2'].T,DZ2);
    DW2 = 1./self.ch['A1'].shape[1] * np.dot(DZ2,self.ch['A1'].T)

    DZ1 = DA1 * self.DerivativeSigmoid( self.ch['Z1'])
    DW1 = 1./self.X.shape[1] * np.dot(DZ1,self.X)
    self.param["W1"] = self.param["W1"] - self.learningrate * DW1
    self.param["W2"] = self.param["W2"] - self.learningrate * DW2
    self.param["W3"] = self.param["W3"] - self.learningrate * DW3
    return;

##Gradient Descent    
  def grad_descent(self,X, Y, iter = 8000):
    
      self.nInit()
      Yh = Y;
      for i in range(0, iter):
          Yh, loss=self.forwardpass()
          self.backwardpass()

          if i % 500 == 0:
            self.loss.append(loss)
      print(Yh)      



In [180]:
cg = ComputationGraphs(training_data,training_labels)
cg.grad_descent(training_data,training_labels)

[[0.00207469 0.00207469 0.00207469 ... 0.00207469 0.00207469 0.00207469]
 [0.00207469 0.00207469 0.00207469 ... 0.00207469 0.00207469 0.00207469]
 [0.00207469 0.00207469 0.00207469 ... 0.00207469 0.00207469 0.00207469]]
